In [1]:
import os, sys
from scipy.optimize import minimize
import scipy.optimize as optimize
import subprocess
import pandas as pd
import numpy as np
import copy

if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')     
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

import traci
import traci.constants as tc    
import socket

def find_free_port():
    s = socket.socket()
    s.bind(('', 0))            # Bind to a free port provided by the host.
    return s.getsockname()[1]  # Return the port number assigned.

In [2]:
# some helper functions

# get sum of all previous list lengths
def get_curr_sum(x):
    if x <= 0:
        sum_len = 0
    else:
        sum_len = 0
        for l in range(x):
            sum_len += len(lst[l])
    return sum_len

# convert list to array 

def list_to_array(input_list):
    flat_list = [item for sublist in input_list for item in sublist]
    flat_list_array = np.asarray(flat_list)
    return flat_list_array

# convert array to list

def array_to_list(old_list, array):

    temp_lst = copy.deepcopy(old_list)
    
    for l in range(len(temp_lst)):
        temp_lst[l] = array[get_curr_sum(l):get_curr_sum(l+1)] + 1
    return temp_lst

def avg_edge_traveltime():
    travel_time_sum = 0
    for j in range(len(all_edge_IDs)):
        #travel_time_sum += traci.edge.getTraveltime(all_edge_IDs[j])
        travel_time_sum += traci.edge.getWaitingTime(all_edge_IDs[j]) 
    return travel_time_sum

def objective_function(phase_array):
    # simulate
    steps = 1000
    traci.load(sumoRldCmd)
    loss = simulate(steps, phase_array)/steps
    print(phase_array)
    return loss

In [3]:
# run me once to start sumo server in seperate command window
simFolder = '2020-09-11-14-47-10'
scaleFactor = 4
port = find_free_port()
sumoBinary = "sumo"
emissFile = "emissions.xml"
tripFile = "all_trips.xml"
sumoServerCmd = [sumoBinary, "--remote-port", str(port),"-c", "/usr/local/Cellar/sumo/1.6.0/share/sumo/tools/"+ simFolder + "/osm.sumocfg", "--emission-output", emissFile, "--tripinfo-output", tripFile, "--scale", str(scaleFactor)]
sumoCmd = [sumoBinary,"-c", "/usr/local/Cellar/sumo/1.6.0/share/sumo/tools/"+ simFolder + "/osm.sumocfg", "--emission-output", emissFile, "--tripinfo-output", tripFile, "--scale", str(scaleFactor)]
sumoRldCmd = ["-c", "/usr/local/Cellar/sumo/1.6.0/share/sumo/tools/"+ simFolder + "/osm.sumocfg", "--emission-output", emissFile, "--tripinfo-output", tripFile, "--scale", str(scaleFactor)]

In [4]:
#os.system(' '.join(sumoServerCmd))
print(' '.join(sumoServerCmd))

sumo --remote-port 55439 -c /usr/local/Cellar/sumo/1.6.0/share/sumo/tools/2020-09-11-14-47-10/osm.sumocfg --emission-output emissions.xml --tripinfo-output all_trips.xml --scale 4


In [ ]:
# convert to csv
#emissCSV = os.path.splitext(emissFile)[0] + ".csv"
#xmlCmd = ["/usr/local/Cellar/sumo/1.6.0/share/sumo/tools/xml/xml2csv.py", emissFile, "-s", "," ,"-o", emissCSV] 
#subprocess.call(' '.join(xmlCmd), shell=True)

In [ ]:
# load with pandas 
#emiss_df = pd.read_csv(emissCSV)

In [ ]:
# convert to csv
#tripCSV = os.path.splitext(tripFile)[0] + ".csv"
#xmlCmd = ["/usr/local/Cellar/sumo/1.6.0/share/sumo/tools/xml/xml2csv.py",tools + "/" + tripFile, "-s", "," ,"-o", tools + "/" + tripCSV] 
#subprocess.call(' '.join(xmlCmd), shell=True)

In [ ]:
# load with pandas 
#trip_df = pd.read_csv(tools + "/" + tripCSV)

In [ ]:
#np.nanmean(trip_df.tripinfo_timeLoss)

                                        ##### Optimization #####

In [5]:
# Connect to to traci
traci.init(port)

# Get traffic lights and Phases
all_traffic_lights = traci.trafficlight.getIDList()
number_of_traffic_lights = traci.trafficlight.getIDCount()
all_edge_IDs = traci.edge.getIDList()

# create empty nested list
lst = [[] for _ in range(number_of_traffic_lights)]
# fill with phase durations    
for i in range(number_of_traffic_lights):
    #print(traci.trafficlight.getCompleteRedYellowGreenDefinition(all_traffic_lights[i]))
    temp = traci.trafficlight.getCompleteRedYellowGreenDefinition(all_traffic_lights[i])
    for j in range(len(temp[0].getPhases())):
        lst[i].append(temp[0].getPhases()[j].duration)

initial_phase_duration_array = list_to_array(lst)

In [6]:
def simulate(SimulationSteps, phase_duration_array):
    phase_duration_list = array_to_list(lst, phase_duration_array)
    step = 0
    
    traci.simulationStep()
    for i in range(number_of_traffic_lights):
        temp = traci.trafficlight.getCompleteRedYellowGreenDefinition(all_traffic_lights[i])
        for k in range(len(phase_duration_list[i])):
            temp[0].getPhases()[k].duration = phase_duration_list[i][k]
            if k == traci.trafficlight.getPhase(all_traffic_lights[i]):
                traci.trafficlight.setPhaseDuration(all_traffic_lights[i],phase_duration_list[i][k])
            #print(temp[0].getPhases()[k].duration)
            
        # set traffic light to new phase durations
        traci.trafficlight.setCompleteRedYellowGreenDefinition(all_traffic_lights[i], temp[0])
        # set to phase 1
        #traci.trafficlight.setPhase(all_traffic_lights[i], 0)
    
    # Simulate 
    
    #junctionID = all_traffic_lights[0]
    #traci.junction.subscribeContext(junctionID, tc.CMD_GET_VEHICLE_VARIABLE, 1000000, [tc.VAR_SPEED, tc.VAR_ALLOWED_SPEED])

    #stepLength = traci.simulation.getDeltaT() 
    #timeLoss = 0
    
    #while step < SimulationSteps:
    while traci.simulation.getMinExpectedNumber() > 0:
    
        traci.simulationStep()
        #scResults = traci.junction.getContextSubscriptionResults(junctionID)
        #halting = 0
        #if scResults:
        #    relSpeeds = [d[tc.VAR_SPEED] / d[tc.VAR_ALLOWED_SPEED] for d in scResults.values()]
            # compute values corresponding to summary-output
        #    running = len(relSpeeds)
        #    halting = len([1 for d in scResults.values() if d[tc.VAR_SPEED] < 0.1])
        #    meanSpeedRelative = sum(relSpeeds) / running
        #    timeLoss += (1 - meanSpeedRelative) * running * stepLength
        #print(traci.simulation.getTime(), timeLoss, halting)
        
        
        step += 1
        #print(traci.trafficlight.getPhase(all_traffic_lights[i]))
    #print(traci.simulation.getTime(), timeLoss, halting)
    SimulTime = traci.simulation.getTime()
    print(SimulTime)
    del temp
    
    #return float(timeLoss) 
    return float(SimulTime)

In [8]:
num = np.random.rand(len(initial_phase_duration_array),1)
new_duration = initial_phase_duration_array + np.transpose(num)*10
print(new_duration)

[[30.3015899  10.09340618 13.94317051 15.85925336  6.00424146 26.84609814
  12.7783576   8.17428373 11.23624494 13.14314762]]


In [10]:
# approx jacobian
fprime = lambda x: optimize.approx_fprime(x, objective_function, 0.01)

In [11]:
# Try Newton-CG

res = minimize(objective_function, new_duration, method = 'Newton-CG', jac = fprime, options = {'disp' : True, 'eps' : 2})

5098.0
[30.3015899  10.09340618 13.94317051 15.85925336  6.00424146 26.84609814
 12.7783576   8.17428373 11.23624494 13.14314762]
5098.0
[30.3015899  10.09340618 13.94317051 15.85925336  6.00424146 26.84609814
 12.7783576   8.17428373 11.23624494 13.14314762]
5098.0
[30.3115899  10.09340618 13.94317051 15.85925336  6.00424146 26.84609814
 12.7783576   8.17428373 11.23624494 13.14314762]
5098.0
[30.3015899  10.10340618 13.94317051 15.85925336  6.00424146 26.84609814
 12.7783576   8.17428373 11.23624494 13.14314762]
5098.0
[30.3015899  10.09340618 13.95317051 15.85925336  6.00424146 26.84609814
 12.7783576   8.17428373 11.23624494 13.14314762]
5098.0
[30.3015899  10.09340618 13.94317051 15.86925336  6.00424146 26.84609814
 12.7783576   8.17428373 11.23624494 13.14314762]
5098.0
[30.3015899  10.09340618 13.94317051 15.85925336  6.01424146 26.84609814
 12.7783576   8.17428373 11.23624494 13.14314762]
5098.0
[30.3015899  10.09340618 13.94317051 15.85925336  6.00424146 26.85609814
 12.778357

In [ ]:
# Try BFGS

res = minimize(objective_function, initial_phase_duration_array, method = 'BFGS', options = {'disp' : True, 'eps' : 2})

5014.0
[26.  5.  4.  6.  4. 26.  5.  4.  6.  4.]
5201.0
[28.  5.  4.  6.  4. 26.  5.  4.  6.  4.]
5047.0
[26.  7.  4.  6.  4. 26.  5.  4.  6.  4.]
5049.0
[26.  5.  6.  6.  4. 26.  5.  4.  6.  4.]
5049.0
[26.  5.  4.  8.  4. 26.  5.  4.  6.  4.]
5048.0
[26.  5.  4.  6.  6. 26.  5.  4.  6.  4.]
5046.0
[26.  5.  4.  6.  4. 28.  5.  4.  6.  4.]
5046.0
[26.  5.  4.  6.  4. 26.  7.  4.  6.  4.]
5055.0
[26.  5.  4.  6.  4. 26.  5.  6.  6.  4.]
5055.0
[26.  5.  4.  6.  4. 26.  5.  4.  8.  4.]
5045.0
[26.  5.  4.  6.  4. 26.  5.  4.  6.  6.]


In [ ]:
#traci.trafficlight.setPhase(all_traffic_lights[i], 7)
traci.trafficlight.getCompleteRedYellowGreenDefinition(all_traffic_lights[0])

In [ ]:
# Result

res

In [ ]:
traci.simulation.saveState('osm_net_save.xml')

In [ ]:
traci.close()

In [ ]:
# get initial travel time

traci.load(sumoRldCmd)
simulate(100, initial_phase_duration_array)
#print(avg_edge_traveltime())


In [ ]:
# new phase duration resimulate

traci.load(sumoRldCmd)
num = np.random.rand(len(initial_phase_duration_array),1)
new_duration = initial_phase_duration_array + np.transpose(num)*60
simulate(100, list_to_array(new_duration))
#print(avg_edge_traveltime())
#traci.load(sumoRldCmd)

In [ ]:
#traci.load(sumoRldCmd)
simulate(100, initial_phase_duration_array)

In [ ]:
num = np.random.rand(len(initial_phase_duration_array),1)
new_duration = initial_phase_duration_array + np.transpose(num)*1000
print(new_duration)

In [ ]:
traci.load(sumoRldCmd)
simulate(100, new_duration)

In [ ]:
traci.close()